
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [1]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool 
from io import StringIO 
import sys


from langchain.schema.runnable import RunnableSequence

In [2]:
# creds.json
# {
# OPENAI_API_KEY: "",
# HUGGINGFACEHUB_API_TOKEN: ""
# }

In [3]:
# Load API Key from a local file (creds.json)
with open("creds.json", 'r') as f:
    creds = json.load(f)
OPENAI_API_KEY = creds.get("OPENAI_API_KEY")

In [4]:
# Initialize the LLM
llm = ChatOpenAI(
    temperature=0.0,  # Set to 0.0 for factual, direct, and deterministic response
    model="gpt-3.5-turbo", 
    api_key=OPENAI_API_KEY 
)

In [5]:
#  Define the Prompt
prompt_text = (
    "Explain Quantum Computing in simple terms. Include a definition, key advantages, "
    "and current research areas. The entire answer must be very concise and "
    "strictly limited to a maximum of 200 characters."
)

#Create the chain/invoke the model (simulated)
response = llm.invoke(prompt_text)
print(response.content)

Quantum computing uses quantum bits (qubits) to perform complex calculations faster than classical computers. Advantages include faster processing and solving complex problems. Research areas include cryptography, optimization, and simulation.


### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [6]:
template = (
    "Explain '{topic}' in simple terms. Include a definition, key advantages, "
    "and current research areas. The entire answer must be very concise and "
    "strictly limited to a maximum of 200 characters. Answer in Ukrainian."
)

# Define the PromptTemplate with the 'topic' variable
prompt_template = PromptTemplate(
    input_variables=["topic"], 
    template=template
)

# Create the chain: PromptTemplate -> LLM
chain: RunnableSequence = prompt_template | llm

In [7]:
topics = [
    "Bayesian methods in ML",  
    "Transformers in ML",       
    "Explainable AI"
]

In [8]:
# Invoke the Model for Each Topic
print(f"Settings: Model=GPT-3.5-Turbo, Temperature=0.0, Max Length=200 chars")
for topic in topics:
    print(f" Processing Topic: '{topic}'")
    try:
        # Invoke the chain, passing the topic as the variable
        response = chain.invoke({"topic": topic})
        output = response.content
        char_count = len(output)
        
        print(f"   Response: {output}")
        print(f"   Char Count: {char_count} / 200 {'✅' if char_count <= 200 else '❌ (Exceeded!)'}")
    except Exception as e:
        print(f"Error calling the model for topic '{topic}'. Check your key and connection: {e}")

Settings: Model=GPT-3.5-Turbo, Temperature=0.0, Max Length=200 chars
 Processing Topic: 'Bayesian methods in ML'
   Response: Байєсівські методи в МО - це підхід, що використовує ймовірність для оцінки невизначеності. Переваги: ефективність, робустність. Дослідження: нейромережі, байєсівські оптимізації.
   Char Count: 179 / 200 ✅
 Processing Topic: 'Transformers in ML'
   Response: Трансформери в машинному навчанні - це модель для обробки послідовних даних. Вони мають високу точність та широке застосування. Досліджуються у покращенні мовного моделювання та обробці природної мови.
   Char Count: 201 / 200 ❌ (Exceeded!)
 Processing Topic: 'Explainable AI'
   Response: Пояснювальний штучний інтелект - це системи, які можуть пояснити свої рішення. Переваги: довіра, зрозумілість. Дослідження: інтерпретація моделей, візуалізація даних.
   Char Count: 166 / 200 ✅




### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [9]:
# !pip install -q langchain_community duckduckgo_search

In [10]:
search_tool = DuckDuckGoSearchRun(name="DuckDuckGo_Search")
tools = [search_tool]

In [11]:
custom_prompt_template = PromptTemplate.from_template("""
You are an expert scientific research assistant. Your task is to find the 5 most recent scientific publications 
on the topic of "Artificial Intelligence".

You have access to the following tools:
{tools} 

Use the tools that are relevant to the user request. You must select one of the following tools: {tool_names}.

Your final answer MUST be a list of exactly 5 publications. 
Each entry in the list MUST contain:
1. Title of the publication.
2. Main authors (or lead author and et al.).
3. A very short snippet/description (1-2 sentences).

Query: {input}
{agent_scratchpad}
""")

In [12]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=custom_prompt_template
)


In [13]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors="Please correct your previous output. Remember your final answer must start with the exact text 'Final Answer:'" 
)

agent_query = "Find the 5 most recent scientific publications on Artificial Intelligence (AI)."
print(f"Agent Query: {agent_query}\n")

Agent Query: Find the 5 most recent scientific publications on Artificial Intelligence (AI).



In [14]:
response = agent_executor.invoke({"input": agent_query}) 
print("FINAL AGENT RESPONSE")
print(response["output"])



> Entering new AgentExecutor chain...
Could not parse LLM output: `1. Title: "A Survey on Artificial Intelligence in Healthcare: Past, Present, and Future Trends"
   Authors: Alaa Abd-Alrazaq, et al.
   This survey paper provides an overview of the applications of artificial intelligence in healthcare, discussing past achievements, current trends, and future directions.

2. Title: "Deep Reinforcement Learning for Multi-Agent Systems: A Review of Challenges, Solutions, and Applications"
   Authors: Shanghang Zhang, et al.
   This review paper explores the challenges, solutions, and applications of deep reinforcement learning in multi-agent systems, highlighting recent advancements in the field.

3. Title: "Interpretable Machine Learning: A Guide for Making Black Box Models Explainable"
   Authors: Christoph Molnar, et al.
   This guide discusses techniques for making black box machine learning models interpretable, providing insights into the importance of explainability in artificial



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [15]:
@tool
def python_repl(code: str) -> str:
    """Executes Python code and returns the stdout output. Useful for mathematical calculations, trend analysis, and data manipulation."""
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    try:
        # Execute the code
        exec(code)
        return redirected_output.getvalue()
    except Exception as e:
        return f"Execution Error: {e}"
    finally:
        # Restore standard output
        sys.stdout = old_stdout


tools = [
    DuckDuckGoSearchRun(name="Internet_Search"),
    python_repl
]

In [16]:
custom_prompt_template = PromptTemplate.from_template("""
You are an expert business analyst specializing in sales forecasting and risk assessment.

You have access to the following tools:
{tools}

Your task is as follows:
1. Analyze the provided historical export data.
2. Use the **Internet_Search** tool to gather relevant external factors: 2025 weather forecasts in Brazil (especially for citrus), global orange demand/prices, and economic projections (inflation/logistics).
3. Use the **python_repl** tool to calculate the historical trend and develop a forecast for 2025, adjusting the base trend based on the external factors you researched.
4. Provide a clear export forecast (in tons) with a concise explanation of the factors that influenced the final adjustment.

You must select one of the following tools: {tool_names}.

Use the following user query:
{input}

{agent_scratchpad}
""")

In [17]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=custom_prompt_template
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors="Please correct your previous output. Your final answer must start with the exact phrase 'Final Answer:', followed immediately by the final forecast and explanation."
)

In [18]:
# User's query containing the historical data
user_query = """
We export oranges from Brazil. We exported 200t in 2021, 190t in 2022, 210t in 2023, and 220t in 2024 (not yet finished). 
Estimate how many oranges we can export in 2025, considering weather conditions in Brazil and global orange demand based on the economic situation.
"""

In [19]:
response = agent_executor.invoke({"input": user_query}) 
print(response["output"])



> Entering new AgentExecutor chain...
Could not parse LLM output: `I will use the python_repl tool to analyze the historical export data and develop a forecast for 2025.

Given the historical export data:
- 2021: 200 tons
- 2022: 190 tons
- 2023: 210 tons
- 2024: 220 tons

I will calculate the trend and forecast for 2025 based on this data.

```python
# Historical export data
exports = [200, 190, 210, 220]

# Calculate the trend
trend = sum(exports) / len(exports)

# Forecast for 2025
forecast_2025 = trend
forecast_2025
```

Based on the historical trend, the forecast for 2025 is approximately 205 tons.

Now, I will consider the external factors by using the Internet_Search tool to gather information on 2025 weather forecasts in Brazil (especially for citrus), global orange demand/prices, and economic projections (inflation/logistics).

After researching the external factors, I will adjust the base trend forecast of 205 tons for 2025. Factors such as favorable weather conditions in B